In [9]:
from dotenv import load_dotenv
load_dotenv()
from langchain_unstructured.document_loaders import UnstructuredLoader
from langchain_community.vectorstores.utils import filter_complex_metadata

file_path = "alain_propos_bonheur.doc"
loader = UnstructuredLoader(
    file_path=file_path,
)
# ベクトルデータベースが対応していないメタデータの削除
documents = filter_complex_metadata(loader.load())

def analysis_langchain_documents(documents, break_point=10):
    num_documents = len(documents)
    lengths = [len(doc.page_content) for doc in documents]
    total_length = sum(lengths)

    print(f"Number of documents: {num_documents}")
    print(f"Total length of text: {total_length} characters")
    print(f'Individual document lengths: {lengths[:10]} {"" if len(lengths) <= 10 else "..."}')

    for i, doc in enumerate(documents):
        print("="*10)
        print(f"Document {i+1} (Length: {len(doc.page_content)}): {doc.page_content[:100]}...")
        print("="*10+"\n\n")
        if i >= break_point:
            break

analysis_langchain_documents(documents, break_point=1000)



INFO: convert /home/syun2/project/alain-chat/alain_propos_bonheur.doc as a Writer document -> /tmp/tmpiv38oznv/alain_propos_bonheur.docx using filter : MS Word 2007 XML


Number of documents: 1029
Total length of text: 318831 characters
Individual document lengths: [39, 523, 142, 34, 28, 150, 32, 32, 37, 49] ...
Document 1 (Length: 39): Alain, Propos sur le bonheur (1928)	199...


Document 2 (Length: 523): Alain (Émile Chartier) (1868-1951) Propos sur le bonheur Un document produit en version numérique pa...


Document 3 (Length: 142): Cette édition électronique a été réalisée par Robert Caron, bénévole, professeur de lettres à la ret...


Document 4 (Length: 34): Alain (Émile Chartier) (1868-1951)...


Document 5 (Length: 28): Propos sur le bonheur (1928)...


Document 6 (Length: 150): Une édition électronique réalisée à partir du livre d’Alain, PROPOS SUR LE BONHEUR. Paris : Éditions...


Document 7 (Length: 32): Polices de caractères utilisée :...


Document 8 (Length: 32): Pour le texte: Times, 12 points....


Document 9 (Length: 37): Pour les citations : Times 10 points....


Document 10 (Length: 49): Pour les notes de bas de page : Times, 10 point

In [10]:
from langchain_chroma import Chroma
from langchain_openai import OpenAIEmbeddings

from dotenv import load_dotenv
import os

# 最初に環境変数を読み込む
load_dotenv()


class VectorDB:
    def __init__(self, db_name):
        self.embeddings = OpenAIEmbeddings(model='text-embedding-3-small')
        self.db = Chroma(persist_directory=db_name, embedding_function=self.embeddings)

    def add_documents(self, documents):
        self.db.add_documents(documents)

    def similarity_search(self, query, **kwargs):
        return self.db.similarity_search(query, **kwargs)
       
    def as_retriever(self, **kwargs):
        return self.db.as_retriever(**kwargs)



In [11]:
vector_db = VectorDB(db_name='vector_db')
vector_db.add_documents(documents)

question = 'Que faire quand on ne se sent pas bien ?'
results = vector_db.similarity_search(question)
display([result.page_content for result in results])

ERROR: Failed to send telemetry event ClientStartEvent: capture() takes 1 positional argument but 3 were given
ERROR: Failed to send telemetry event ClientCreateCollectionEvent: capture() takes 1 positional argument but 3 were given
INFO: HTTP Request: POST https://api.openai.com/v1/embeddings "HTTP/1.1 200 OK"
INFO: HTTP Request: POST https://api.openai.com/v1/embeddings "HTTP/1.1 200 OK"
INFO: HTTP Request: POST https://api.openai.com/v1/embeddings "HTTP/1.1 200 OK"


["Le remède\xa0? Fuir sa famille. Aller vivre au milieu d'indifférents, qui vous demanderont d'un air distrait\xa0: «\xa0Comment vous portez-vous\xa0?», mais s'enfuiront si vous répondez sérieusement\xa0; de gens qui n'écouteront pas vos plaintes et ne poseront pas sur vous ce regard chargé de tendre sollicitude qui vous étranglait l'estomac. Dans ces conditions, si vous ne tombez pas tout de suite dans le désespoir, vous guérirez. Morale\xa0: ne dites jamais à quelqu'un qu'il a mauvaise mine.",
 "Le remède\xa0? Fuir sa famille. Aller vivre au milieu d'indifférents, qui vous demanderont d'un air distrait\xa0: «\xa0Comment vous portez-vous\xa0?», mais s'enfuiront si vous répondez sérieusement\xa0; de gens qui n'écouteront pas vos plaintes et ne poseront pas sur vous ce regard chargé de tendre sollicitude qui vous étranglait l'estomac. Dans ces conditions, si vous ne tombez pas tout de suite dans le désespoir, vous guérirez. Morale\xa0: ne dites jamais à quelqu'un qu'il a mauvaise mine."

In [14]:
from langchain_core.output_parsers import StrOutputParser
from langchain_core.prompts import ChatPromptTemplate
from langchain_core.runnables import RunnablePassthrough
from langchain_openai import ChatOpenAI

def create_simple_rag_chain(vector_db):
    prompt = ChatPromptTemplate.from_template('''
    質問: {question}

    情報源: """{context}
    """
    ''')

    llm = ChatOpenAI(model='gpt-4o-mini', temperature=0.1)
    retriever = vector_db.as_retriever()

    chain = {
        'question': RunnablePassthrough(),
        'context': retriever
    } | prompt | llm | StrOutputParser()
    return chain

In [22]:
def create_multilingual_rag_chain(vector_db):
    # 質問翻訳用のチェーン
    translation_prompt = ChatPromptTemplate.from_template('''
    以下の日本語の質問をフランス語に翻訳してください。翻訳結果のみを返してください。
    
    質問: {question}
    ''')
    
    # 回答生成用のプロンプト
    answer_prompt = ChatPromptTemplate.from_template('''
    以下のフランス語の情報源を参考にして、日本語で質問に答えてください。
    
    元の質問（日本語）: {original_question}
    
    情報源（フランス語）:
    {context}
    
    回答は日本語で、情報源の内容を正確に反映してください。
    ''')

    llm = ChatOpenAI(model='gpt-4o-mini', temperature=0.1)
    retriever = vector_db.as_retriever(search_kwargs={"k": 3})

    # 質問を翻訳してから検索
    chain = {
        'original_question': RunnablePassthrough(),
        'translated_question': translation_prompt | llm | StrOutputParser(),
    } | {
        'original_question': lambda x: x['original_question'],
        'context': lambda x: retriever.invoke(x['translated_question'])
    } | answer_prompt | llm | StrOutputParser()
    
    return chain

In [ ]:
def create_multilingual_rag_chain(vector_db):
    # 質問翻訳用のチェーン
    translation_prompt = ChatPromptTemplate.from_template('''
    以下の日本語の質問をフランス語に翻訳してください。翻訳結果のみを返してください。
    
    質問: {question}
    ''')
    
    # 回答生成用のプロンプト
    answer_prompt = ChatPromptTemplate.from_template('''
    以下のフランス語の情報源を参考にして、日本語で質問に答えてください。
    
    元の質問（日本語）: {original_question}
    
    情報源（フランス語）:
    {context}
    
    回答は日本語で、情報源の内容を正確に反映してください。
    ''')

    llm = ChatOpenAI(model='gpt-4o-mini', temperature=0.1)
    retriever = vector_db.as_retriever(search_kwargs={"k": 3})

    # シンプルな修正版
    chain = (
        {"question": RunnablePassthrough()}
        | RunnablePassthrough.assign(
            translated_question=translation_prompt | llm | StrOutputParser()
        )
        | RunnablePassthrough.assign(
            original_question=lambda x: x["question"],
            context=lambda x: retriever.invoke(x["translated_question"])
        )
        | answer_prompt 
        | llm 
        | StrOutputParser()
    )
    
    return chain

In [26]:
import langchain
langchain.verbose = False
langchain.debug = False
langchain.llm_cache = False

question = '気分をよくするためにはどうすればいいですか？'
chain = create_multilingual_rag_chain(vector_db)
result = chain.invoke(question)
print(result)

INFO: HTTP Request: POST https://api.openai.com/v1/chat/completions "HTTP/1.1 200 OK"
INFO: HTTP Request: POST https://api.openai.com/v1/embeddings "HTTP/1.1 200 OK"
INFO: HTTP Request: POST https://api.openai.com/v1/chat/completions "HTTP/1.1 200 OK"


気分をよくするためには、家族から離れて無関心な人々の中で生活することが一つの方法です。無関心な人々は、あなたに「元気ですか？」と気軽に尋ねるかもしれませんが、真剣に答えると逃げてしまいます。彼らはあなたの不満を聞かず、あなたに対して過度な関心を持たないため、気持ちが楽になるかもしれません。このような環境で、すぐに絶望に陥らなければ、気分は改善されるでしょう。重要な教訓は、誰かに「あなたは元気がない」と言わないことです。


In [21]:
import langchain
langchain.verbose = False
langchain.debug = False
langchain.llm_cache = False

question = 'Que faire quand on ne se sent pas bien ?'
chain = create_simple_rag_chain(vector_db)
result = chain.invoke(question)
print(result)


INFO: HTTP Request: POST https://api.openai.com/v1/embeddings "HTTP/1.1 200 OK"
INFO: HTTP Request: POST https://api.openai.com/v1/chat/completions "HTTP/1.1 200 OK"


Quand on ne se sent pas bien, il existe plusieurs approches que l'on peut envisager pour améliorer son état. Voici quelques suggestions :

1. **Éloignement des sources de stress** : Parfois, s'éloigner de son environnement habituel, comme sa famille ou des personnes qui suscitent des émotions négatives, peut aider. Cela permet de se retrouver dans un cadre où l'on se sent moins jugé et plus libre d'exprimer ses sentiments sans crainte de réaction.

2. **Prendre du temps pour soi** : S'accorder des moments de solitude ou de réflexion peut être bénéfique. Cela peut inclure des activités relaxantes comme la lecture, la méditation ou simplement se promener dans la nature.

3. **Parler à quelqu'un** : Si vous vous sentez à l'aise, partager vos sentiments avec un ami ou un professionnel peut apporter un soulagement. Parfois, le simple fait de verbaliser ses pensées aide à les clarifier et à les atténuer.

4. **Se concentrer sur des activités positives** : S'engager dans des activités qui vou